Give me a python code for question answering using roberta model. In the model embed a context from few cyrillic pdf files in pdf_files directory, few cyrillic HTML files in html_files directory and a questions-answers.csv in csv_files directory. After this receive a question and retrieve an answer using INSAIT BGGPT 9B model.

In [ ]:
import os
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from bs4 import BeautifulSoup
import pdfplumber
import torch

# Load RoBERTa QA model and tokenizer
model_name_roberta = "deepset/roberta-base-squad2"
tokenizer_roberta = AutoTokenizer.from_pretrained(model_name_roberta)
model_roberta = AutoModelForQuestionAnswering.from_pretrained(model_name_roberta)
qa_pipeline_roberta = pipeline("question-answering", model=model_roberta, tokenizer=tokenizer_roberta)

# Load the INSAIT BGGPT 9B model and tokenizer (assuming it exists on Hugging Face, if not, you'll need to adjust)
model_name_bggpt = "INSAIT/BGGPT-9B"
tokenizer_bggpt = AutoTokenizer.from_pretrained(model_name_bggpt)
model_bggpt = AutoModelForQuestionAnswering.from_pretrained(model_name_bggpt)

# Function to extract text from PDFs
def extract_text_from_pdfs(directory):
    text = ""
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
    return text

# Function to extract text from HTML files
def extract_text_from_htmls(directory):
    text = ""
    for filename in os.listdir(directory):
        if filename.endswith(".html") or filename.endswith(".htm"):
            html_path = os.path.join(directory, filename)
            with open(html_path, "r", encoding="utf-8") as html_file:
                soup = BeautifulSoup(html_file, "html.parser")
                text += soup.get_text() + "\n"
    return text

# Function to extract context from CSV files
def extract_context_from_csv(directory):
    text = ""
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            csv_path = os.path.join(directory, filename)
            df = pd.read_csv(csv_path)
            for _, row in df.iterrows():
                text += f"Q: {row['question']} A: {row['answer']}\n"
    return text

# Function to process context
def preprocess_context(pdf_dir, html_dir, csv_dir):
    pdf_context = extract_text_from_pdfs(pdf_dir)
    html_context = extract_text_from_htmls(html_dir)
    csv_context = extract_context_from_csv(csv_dir)
    combined_context = pdf_context + html_context + csv_context
    return combined_context

# Function to get an answer from RoBERTa model
def get_answer_roberta(question, context):
    result = qa_pipeline_roberta(question=question, context=context)
    return result

# Function to get an answer from INSAIT BGGPT 9B model (this step assumes it can handle a similar format)
def get_answer_bggpt(question, context):
    # Using the BGGPT model for answering the question
    inputs = tokenizer_bggpt(question, context, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model_bggpt(**inputs)
        answer = tokenizer_bggpt.decode(outputs.logits.argmax(-1), skip_special_tokens=True)
    return answer

# Main function
if __name__ == "__main__":
    # Directories for the files
    pdf_dir = "pdf_files"
    html_dir = "html_files"
    csv_dir = "csv_files"

    # Process and combine the context from files
    print("Extracting and combining context from files...")
    context = preprocess_context(pdf_dir, html_dir, csv_dir)

    # Start the question-answering loop
    print("\nContext loaded. You can now ask questions.")
    while True:
        question = input("Enter your question (or type 'exit' to quit): ").strip()
        if question.lower() == "exit":
            print("Goodbye!")
            break

        # Get answer using RoBERTa model first for context-based extraction
        print("\nAnswering using RoBERTa model for context extraction...")
        roberta_answer = get_answer_roberta(question, context)
        print(f"RoBERTa Answer: {roberta_answer['answer']}\nConfidence: {roberta_answer['score']:.2f}")
        
        # Get answer using INSAIT BGGPT 9B model for detailed answering
        print("\nAnswering using INSAIT BGGPT 9B model...")
        bggpt_answer = get_answer_bggpt(question, context)
        print(f"BGGPT 9B Answer: {bggpt_answer}\n")


### Explanation:
#### Context Extraction:

The code extracts context from files located in the specified directories:
PDF files are processed using the pdfplumber library to extract text.
HTML files are processed using BeautifulSoup to get plain text.
CSV files are read with pandas and formatted for question-answer extraction.
RoBERTa QA Model:

The deepset/roberta-base-squad2 model is used to extract context-based answers. It operates by finding the most relevant span of text within the provided context.

#### INSAIT BGGPT 9B Model:

After extracting context, the script uses the INSAIT BGGPT 9B model (which assumes it exists or is available via Hugging Face). This model is used to provide a more advanced, ChatGPT-style response after the context extraction.

#### Question-Answering Loop:

The user is prompted to enter a question, and the system first answers using the RoBERTa model, followed by answering with the BGGPT 9B model.

##### Notes:
BGGPT 9B: This code assumes that INSAIT BGGPT 9B exists on Hugging Face and can be accessed using the standard model loading process. If the model is not available or requires specific API access, you may need to adapt the model loading part accordingly.

Performance Considerations: RoBERTa is used first for quick context extraction, followed by a more sophisticated answer using BGGPT 9B. Make sure you have adequate system resources for running large models like BGGPT 9B, especially if they are not pre-optimized for inference.

##### Setup Requirements:
###### Libraries:

pip install transformers pandas bs4 pdfplumber torch
Directory Structure:

Ensure that the pdf_files, html_files, and csv_files directories exist and contain the relevant files.

###### Running the Script: Execute the script:

#### python question_answering_with_roberta_bggpt.py

This should provide you with a framework that extracts context from multiple sources and answers questions using both RoBERTa and the INSAIT BGGPT 9B model.